In [ ]:
import sys
import os
from pathlib import Path

current_dir = str(Path(os.getcwd()).parent.parent)
print(f"Current dir: {current_dir}")
sys.path.insert(0, current_dir)

In [ ]:
import numpy as np
from step03training.full_data.data_utils import (
    load_training_data,
    filter_unused_features,
    add_interaction_features,
    load_training_scores,
)

from shared.paths import vectors_file, scores_file, filtered_data

print("--- Step 1: Loading Data ---")
if not os.path.exists(filtered_data):
    x, y = load_training_data(vectors_file, scores_file)
else:
    x = np.array([])
    y = load_training_scores(scores_file)


print(f"Loaded Data Shape: {x.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
x, kept_indices = filter_unused_features(x, y)
print(f"Filtered Data Shape: {x.shape}")

print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
x, _ = add_interaction_features(x, y, target_k=200)
print(f"Final Data Shape (with Interactions): {x.shape}")
print("Data Preparation Complete.")

In [ ]:
import random
from IPython.display import clear_output
from shared.config import config
from step03training.full_data.run import optimize_hyperparameters, evaluate_hyperparameter_combo
from step03training.full_data.config_utils import (
    crossover_config,
    generate_random_config,
    #generate_slowest_setup
)
from shared.paths import training_output_dir

# Setup Loop Variables
current_cfg = generate_random_config()
top_cfg = config["training"]["top"]
fastest_cfg = config["training"]["fastest"]
slowest_cfg = config["training"]["slowest"]
#slowest_cfg=generate_slowest_setup()
#config["training"]["slowest"]= slowest_cfg
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

temp_model_base = os.path.join(training_output_dir, "temp_model")

get_base_score=True
if get_base_score:
    for value in ["top","fastest"]:
        print(f"getting {value} base score ...")
        current=config["training"][value]
        score, t_time = evaluate_hyperparameter_combo(current , temp_model_base, X=x, y=y)
        new_current = {**current , "best_score": score, "training_time": t_time}
        current.update(new_current)
        print (f"score: {score}")



for i in range(max_iters):
    if i%10==0 and i>0:
        clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]

    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.01:
        base_cfg = current_cfg
        strategy = "RANDOM_START"
    elif rand_val < 0.30:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.9:
        base_cfg = top_cfg
        strategy = "TOP"
    #elif rand_val < 0.80:
    else:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"
    # else:
    #     base_cfg = slowest_cfg
    #     strategy = "SLOWEST"

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")
    print (f"current score; {base_cfg["best_score"]}")
    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=x, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)